In [1]:
import multiprocessing
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import os
import neuro_morpho_toolbox as nmt


def readSWC(swc_path, mode='simple'):
    n_skip = 0
    with open(swc_path, "r") as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith("#"):
                n_skip += 1
            else:
                break
    names = ["##n", "type", "x", "y", "z", "r", "parent"]
    used_cols = [0, 1, 2, 3, 4, 5, 6]
    if mode == 'simple':
        pass
    df = pd.read_csv(swc_path, index_col=0, skiprows=n_skip, sep=" ",
                     usecols=used_cols,
                     names=names
                     )

    return df


def get_node_region(point):
    p = point[['x', 'y', 'z']].copy()
    p['x'] = p['x'] / nmt.annotation.space['x']
    p['y'] = p['y'] / nmt.annotation.space['y']
    p['z'] = p['z'] / nmt.annotation.space['z']
    p = p.round(0).astype(int)
    if ((p.x.iloc[0] >= 0) & (p.x.iloc[0] < nmt.annotation.size['x']) &
            (p.y.iloc[0] >= 0) & (p.y.iloc[0] < nmt.annotation.size['y']) &
            (p.z.iloc[0] >= 0) & (p.z.iloc[0] < nmt.annotation.size['z'])
    ):
        region_id = nmt.annotation.array[p.x.iloc[0],
                                         p.y.iloc[0],
                                         p.z.iloc[0]]
        if region_id in list(nmt.bs.dict_to_selected.keys()):
            region_id = nmt.bs.dict_to_selected[region_id]
            region = nmt.bs.id_to_name(region_id)
            return region

    return 'unknow'

In [2]:
src=r'D:\datasets\Allen_Janelia\Allen_Jan'
# src=r'D:\datasets\Allen_Janelia\Allen_Jan_re10um'
swc_list = os.listdir(src)
swc_path_list = [src + '\\' + i for i in swc_list]
swc_dict = {}
for cur_swc_path in tqdm(swc_path_list):
    swc_dict[cur_swc_path.split('\\')[-1]] = readSWC(cur_swc_path)


100%|██████████████████████████████████████████████████████████████████████████████| 3001/3001 [01:49<00:00, 27.34it/s]


In [3]:
df = swc_dict['17302_00036.swc']
soma = df[df.type==1]
soma

,type,x,y,z,r,parent
##n,,,,,,
1,1,8208.525,3948.425,7956.225,1.0,-1


In [8]:
soma.drop_duplicates(['x','y','z'])

,type,x,y,z,r,parent
##n,,,,,,
1,1,8208.525,3948.425,7956.225,1.0,-1


In [4]:
soma_dict = {}
for k,v in tqdm(swc_dict.items()):
    soma = v[(v.type==1) & (v.parent==-1)]
    soma = soma.drop_duplicates(['x','y','z'])
    if len(soma)!=1:
        print(k)
    else:
        soma_dict[k] = soma
        
# del swc_dict

100%|█████████████████████████████████████████████████████████████████████████████| 3001/3001 [00:04<00:00, 703.30it/s]


In [ ]:
# soma_dict['15257_2226_x16029_y23953.swc']
# soma_dict

In [5]:
soma_df = pd.DataFrame(columns=soma_dict[list(soma_dict.keys())[0]].columns)
for k,v in soma_dict.items():
    v.index = [k]
    soma_df = pd.concat([soma_df, v], axis=0)
    
soma_df

,type,x,y,z,r,parent
17109_1701_x8048_y22277.swc,1,2820.175,3990.050,7795.550,1.0,-1
17109_1801_x6698_y12550.swc,1,3003.125,3453.775,4191.800,1.0,-1
17109_1901_x9602_y10508.swc,1,3183.950,4764.250,3375.825,1.0,-1
17109_2201_x8046_y23301.swc,1,3479.625,3790.725,8126.500,1.0,-1
17109_2301_x8535_y23051.swc,1,3562.350,4013.725,8007.975,1.0,-1
...,...,...,...,...,...,...
pre_18866_00186.swc,1,9014.500,3195.725,2810.000,1.0,-1
pre_18866_00299.swc,1,8793.700,4237.000,2382.125,1.0,-1
pre_18866_01458.swc,1,8035.300,2626.400,7929.675,1.0,-1
pre_18868_00443.swc,1,3882.700,2516.325,7802.225,1.0,-1


In [14]:
# soma_df.to_csv('3002_cell_soma.csv', sep=',')
soma_df = pd.read_csv('../final_data/3002_cell_soma.csv', sep=',', index_col=0)
soma_df

,type,x,y,z,r,parent
17109_1701_x8048_y22277.swc,1,2820.175,3990.050,7795.550,1.0,-1
17109_1801_x6698_y12550.swc,1,3003.125,3453.775,4191.800,1.0,-1
17109_1901_x9602_y10508.swc,1,3183.950,4764.250,3375.825,1.0,-1
17109_2201_x8046_y23301.swc,1,3479.625,3790.725,8126.500,1.0,-1
17109_2301_x8535_y23051.swc,1,3562.350,4013.725,8007.975,1.0,-1
...,...,...,...,...,...,...
pre_18866_00186.swc,1,9014.500,3195.725,2810.000,1.0,-1
pre_18866_00299.swc,1,8793.700,4237.000,2382.125,1.0,-1
pre_18866_01458.swc,1,8035.300,2626.400,7929.675,1.0,-1
pre_18868_00443.swc,1,3882.700,2516.325,7802.225,1.0,-1


In [27]:
soma_df['somaRegion'] = soma_df.apply(lambda r: 
            get_node_region(pd.DataFrame({'x':[r.x], 'y':[r.y], 'z':[r.z]})), 
            axis=1)

In [34]:
regions_counts = soma_df['somaRegion'].value_counts()
sele_regions = regions_counts[regions_counts>30]
sele_regions_list = [i for i in sele_regions.index]
sele_regions_list.remove('unknow')
sele_regions_list.remove('fiber tracts')
sele_regions_list

['MOs',
 'VPM',
 'CP',
 'MOp',
 'VPL',
 'SUB',
 'DG',
 'SSs',
 'SSp-m',
 'PRE',
 'LGd',
 'SSp-bfd',
 'VAL',
 'SSp-ul',
 'RT',
 'AId',
 'ACAd',
 'MG',
 'ZI']

In [40]:
tmp_df = soma_df[soma_df.somaRegion.isin(sele_regions_list)]
tmp_df.to_csv('D:\\2023\\morphology_embedding\\raw_dataset\\select_neuron.csv', sep=',')
tmp_df

,type,x,y,z,r,parent,somaRegion
17109_1701_x8048_y22277.swc,1,2820.175,3990.050,7795.550,1.0,-1,AId
17109_2201_x8046_y23301.swc,1,3479.625,3790.725,8126.500,1.0,-1,MOp
17109_2301_x8535_y23051.swc,1,3562.350,4013.725,8007.975,1.0,-1,AId
17109_2301_x9418_y23665.swc,1,3535.425,4363.250,8328.100,1.0,-1,AId
17109_2401_x8977_y24184.swc,1,3737.100,4093.525,8497.250,1.0,-1,AId
...,...,...,...,...,...,...,...
pre_18866_00186.swc,1,9014.500,3195.725,2810.000,1.0,-1,DG
pre_18866_00299.swc,1,8793.700,4237.000,2382.125,1.0,-1,DG
pre_18866_01458.swc,1,8035.300,2626.400,7929.675,1.0,-1,DG
pre_18868_00443.swc,1,3882.700,2516.325,7802.225,1.0,-1,MOp


In [9]:
src = r'D:\datasets\Allen_Janelia\Allen_Jan_re10um'
swc_list = os.listdir(src)

In [10]:
df = readSWC(src+'\\17109_1701_x8048_y22277.swc')
df['type'].unique()

array([1, 4, 3, 2], dtype=int64)

In [11]:
denRadius_dict = {}
for swc in tqdm(swc_list):
    swc_path=src+'\\'+swc
    df = readSWC(swc_path)
    child = df[df.parent!=-1]
    parent = df.loc[child.parent]
    tips = df.drop(parent.index)
    tips = tips[tips.type.isin([3,4])]
    soma = soma_df.loc[[swc]]
    if len(soma)!=1:
        print(swc)
        continue
    dis_ = tips[['x', 'y', 'z']].values - soma[['x','y','z']].values
    dis = np.sqrt((dis_*dis_).sum(axis=1))
    if len(dis) == 0:
        print(swc)
        continue
    den_radius = np.average(dis)
    denRadius_dict[swc.split('.')[0]] = den_radius

 63%|█████████████████████████████████████████████████▏                            | 1894/3001 [00:19<00:16, 68.58it/s]

AA0114.swc
AA0115.swc


 73%|████████████████████████████████████████████████████████▊                     | 2184/3001 [00:23<00:12, 65.31it/s]

AA0411.swc


 75%|██████████████████████████████████████████████████████████▋                   | 2258/3001 [00:24<00:09, 81.25it/s]

...

 82%|███████████████████████████████████████████████████████████████▊              | 2454/3001 [00:26<00:06, 81.08it/s]

AA0670.swc
AA0672.swc


 84%|█████████████████████████████████████████████████████████████████▊            | 2533/3001 [00:27<00:06, 67.04it/s]

AA0754.swc
AA0763.swc


100%|██████████████████████████████████████████████████████████████████████████████| 3001/3001 [00:32<00:00, 91.46it/s]


In [12]:
tmp = pd.DataFrame({'swc_id':list(denRadius_dict.keys()), 'denRadius':list(denRadius_dict.values())})
tmp.to_csv('denRadius.csv', sep=',', index=None)
tmp

,swc_id,denRadius
0,17109_1701_x8048_y22277,119.125794
1,17109_1801_x6698_y12550,135.610506
2,17109_1901_x9602_y10508,135.098438
3,17109_2201_x8046_y23301,141.924331
4,17109_2301_x8535_y23051,159.620525
...,...,...
2984,pre_18866_00186,257.409706
2985,pre_18866_00299,259.475274
2986,pre_18866_01458,352.579915
2987,pre_18868_00443,312.031726


In [29]:
swc = '17109_1701_x8048_y22277.swc'
swc_path=src+'\\'+swc
df = readSWC(swc_path)
child = df[df.parent!=-1]
parent = df.loc[child.parent]
tips = df.drop(parent.index)
tips = tips[tips.type.isin([3,4])]
soma = soma_df.loc[[swc]]
if len(soma)!=1:
    print(swc)
    
dis_ = tips[['x', 'y', 'z']].values - soma[['x','y','z']].values
dis = np.sqrt((dis_*dis_).sum(axis=1))
if len(dis) == 0:
    print(swc)
    
den_radius = np.average(dis)

In [30]:
df[df.parent==14]

,type,x,y,z,r,parent
##n,,,,,,


In [31]:
soma

,type,x,y,z,r,parent
17109_1701_x8048_y22277.swc,1,2820.175,3990.05,7795.55,1.0,-1


In [32]:
tips

,type,x,y,z,r,parent
##n,,,,,,
14,4,2772.80,3865.38,7795.88,1,15
19,4,2755.57,3823.23,7849.92,1,20
39,4,2714.85,3866.73,7832.17,1,40
44,4,2725.27,3899.23,7811.20,1,45
46,4,2736.62,3902.75,7821.27,1,47
67,4,2686.23,3805.75,7665.23,1,68
77,4,2740.38,3808.43,7657.17,1,78
85,4,2734.32,3868.45,7728.58,1,86
92,4,2769.40,3921.75,7689.08,1,93


In [35]:
print(47.375**2+124.67**2+0.33**2)
print(np.sqrt(47.375**2+124.67**2+0.33**2))
dis_

17787.108425
133.36831867051484


array([[ -47.375, -124.67 ,    0.33 ],
       [ -64.605, -166.82 ,   54.37 ],
       [-105.325, -123.32 ,   36.62 ],
       [ -94.905,  -90.82 ,   15.65 ],
       [ -83.555,  -87.3  ,   25.72 ],
       [-133.945, -184.3  , -130.32 ],
       [ -79.795, -181.62 , -138.38 ],
       [ -85.855, -121.6  ,  -66.97 ],
       [ -50.775,  -68.3  , -106.47 ],
       [  -2.195, -166.3  ,  -73.22 ],
       [  15.445, -108.05 , -110.57 ],
       [  43.755,  -44.57 ,  -65.5  ],
       [  59.275,  -44.6  ,  -72.1  ],
       [  84.825,  -37.07 ,  -33.4  ],
       [  46.005, -122.93 ,   14.3  ],
       [  52.325,   39.8  ,  -96.35 ],
       [  69.325,   69.9  ,  -70.13 ],
       [ 116.575,   35.07 ,  -48.9  ],
       [  63.375,  -59.03 , -113.63 ],
       [ 111.075,   22.43 ,  -50.28 ],
       [ 105.775,   13.68 ,  -42.82 ],
       [ -30.575,   34.93 ,  -54.2  ],
       [ -34.625,   61.9  ,  -57.45 ],
       [ -50.605,   24.1  ,  -38.38 ],
       [ -17.245,   80.4  ,  -22.97 ],
       [  41.445,   63.6 

In [33]:
dis

array([133.36831867, 186.97276627, 166.25944312, 132.28814733,
       123.54855898, 262.47105636, 241.87205466, 163.2256779 ,
       136.30426085, 181.71867385, 155.36754946,  90.50544141,
       103.4461001 ,  98.41252728, 132.03308269, 116.64162261,
       120.87258798, 131.19011215, 142.87305003, 123.97112133,
       114.93061135,  71.36228363,  91.27432895,  67.93158636,
        85.37664157,  77.58613874, 118.15604354, 138.98714482,
       105.55398631,  89.46354523, 111.57809339, 130.76097822,
       107.07858528,  86.89744314, 126.18371458,  90.34238222,
        88.10258183,  79.31770247,  87.8532801 ,  94.45830257,
        97.25113585,  72.81438268,  99.37524101, 100.51051251,
        84.30786752])